# Chains in LangChain

## Outline

- LLMChain
- Sequential Chains
  - SimpleSequentialChain
  - SequentialChain
- Router Chain


In [ ]:
# !pip install langchain langchain-fireworks pandas python-dotenv langchain_community

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.


In [ ]:
# account for deprecation of LLM model
import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [ ]:
import pandas as pd

df = pd.read_csv("Data.csv")

In [ ]:
df.head()

## Simple Chain


In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm_model = "accounts/fireworks/models/llama-v3p1-70b-instruct"
llm = ChatFireworks(temperature=0, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=False
)

In [ ]:
print(overall_simple_chain.run(product))

New API


In [ ]:
from langchain.schema.output_parser import StrOutputParser

overall_simple_chain = (
    first_prompt | llm | StrOutputParser() | second_prompt | llm | StrOutputParser()
)
print(overall_simple_chain.invoke(product))

## RunablePassThrough, RunableParallel, RunableLambda


In [ ]:
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableLambda,
    RunnableParallel,
)

In [ ]:
chain = RunnablePassthrough() | RunnablePassthrough() | RunnablePassthrough()
chain.invoke("hello")

In [ ]:
def input_to_upper(input: str):
    output = input.upper()
    return output

In [ ]:
chain = RunnablePassthrough() | RunnableLambda(input_to_upper) | RunnablePassthrough()
chain.invoke("hello")

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})

In [ ]:
chain.invoke("hello")

In [ ]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": lambda z: z["input2"]})

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

### Nested chains - now it gets more complicated!


In [ ]:
def find_keys_to_uppercase(input: dict):
    output = input.get("input", "not found").upper()
    return output

In [ ]:
chain = RunnableParallel({
    "x": RunnablePassthrough() | RunnableLambda(find_keys_to_uppercase),
    "y": lambda z: z["input2"],
})

In [ ]:
chain.invoke({"input": "hello", "input2": "goodbye"})

## A more complicated chain


Deprecated API


In [ ]:
# Deprecated API
from langchain.chains import SequentialChain

llm = ChatFireworks(temperature=0, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:" "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:" "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True,
)

In [ ]:
review = df.Review[5]
results_old = overall_chain(review)
results_old

New API for RunableParrallel


In [ ]:
from langchain_core.runnables.base import RunnableParallel
from langchain_core.tracers.stdout import ConsoleCallbackHandler

In [ ]:
first_chain = first_prompt | llm | StrOutputParser()
second_chain = second_prompt | llm | StrOutputParser()
third_chain = third_prompt | llm | StrOutputParser()
fourth_chain = fourth_prompt | llm | StrOutputParser()

overall_chain = (
    first_chain
    | {
        "summary": second_chain,
        "language": third_chain,
    }
    | fourth_chain
)
# Invoke the overall chain with the input review
result = overall_chain.invoke(
    {"Review": review},
)  # config={"callbacks": [ConsoleCallbackHandler()]})


In [ ]:
print(result)

In [ ]:
print(results_old["followup_message"])

## Router Chain(deprecated) , use combination of all 3 Runables


In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template,
    },
]

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

llm = ChatFireworks(temperature=0, model=llm_model)

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2? Think step by step")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

Use Combination of Runable


In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = prompt | llm
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
destination_chains.keys()

In [ ]:
print(destinations_str)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from typing_extensions import TypedDict
from typing import Literal
from operator import itemgetter

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
route_prompt = PromptTemplate.from_template(template=router_template)


In [ ]:
class RouteQuery(TypedDict):
    """Route query to destination."""

    destination: Literal["physics", "math", "History", "computer science"]


route_chain = (
    router_prompt | llm.with_structured_output(RouteQuery) | itemgetter("destination")
)


def condition_check(x, chain):
    return chain[x["destination"]]


chain = (
    {
        "destination": route_chain,  # "animal" or "vegetable"
        "input": RunnablePassthrough(),  # pass through input query
    }
    | RunnableLambda(
        # if animal, chain_1. otherwise, chain_2.
        lambda x: condition_check(x, destination_chains)
    )
    | StrOutputParser()
)

In [ ]:
print(chain.invoke("What is 2-3*5 +2"))

In [ ]:
print(chain.invoke("What is black body radiation?"))

Try another way


In [ ]:
# Another way
from langchain.output_parsers.json import SimpleJsonOutputParser

route_chain = (
    router_prompt
    | llm  # .with_structured_output(RouteQuery)
    | SimpleJsonOutputParser()  # itemgetter("destination")
)


def condition_check(x, destination_chains):
    return destination_chains[x["classifier"]["destination"]]


chain = (
    RunnableParallel(
        {
            "classifier": route_chain,
            "input": RunnablePassthrough(),
        },  # pass through input query
    )
    | RunnableLambda(
        # if animal, chain_1. otherwise, chain_2.
        lambda x: condition_check(x, destination_chains)
    )
    | StrOutputParser()
)

In [ ]:
print(chain.invoke("What is 2-3*5 +2"))

In [ ]:
print(chain.invoke("What is black body radiation?"))